In [37]:
# Standard imports
import pandas as pd
import numpy as np

In [38]:
# Load in data from the excel file.
file_name = 'H-2B_Disclosure_Data_FY2019.xlsx'
raw_data = pd.read_excel(file_name, header=0)

print('Number of rows before beginning data cleaning:', raw_data.shape[0])
print('Number of columns before beginning data cleaning:', raw_data.shape[1])
raw_data.head()

Number of rows before beginning data cleaning: 8072
Number of columns before beginning data cleaning: 62


,CASE_NUMBER,DECISION_DATE,VISA_CLASS,SUBMIT_DATE_NEW,CASE_STATUS,CERTIFICATION_BEGIN_DATE,CERTIFICATION_END_DATE,EMPLOYER_NAME,TRADE_NAME_DBA,EMPLOYER_ADDRESS_1,...,WORKSITE_STATE,WORKSITE_POSTAL_CODE,OTHER_WORKSITE_LOCATION,SWA_NAME,JOB_IDNUMBER,JOB_START_DATE,JOB_END_DATE,WORKSITE_ADDRESS_1,WORKSITE_ADDRESS_2,FEIN
0,H-400-18198-951583,2018-10-01,H-2B,2018-08-10 17:42:18.943,Determination Issued - Certification,2018-10-24,2018-11-30,"SIMONS PROPERTY MAINTENANCE, INC.",NaN,3542 Boulden Blvd.,...,UTAH,84093,Y,NaN,NaN,NaT,NaT,2309 EAST 9400 SOUTH,NaN,870581683.0
1,H-400-18199-575646,2018-10-01,H-2B,2018-08-31 11:34:26.506,Determination Issued - Certification,2018-11-14,2019-05-01,"Clay Lowry Forestry, Inc.",NaN,2848 Highway 8 S,...,ARKANSAS,71647,Y,NaN,NaN,NaT,NaT,2848 Hwy. 8 South,NaN,621714069.0
2,H-400-18215-159343,2018-10-01,H-2B,2018-08-17 08:58:25.234,Determination Issued - Partial Certification,2018-11-15,2019-04-01,"Pritchett Forestry Services, Inc.",NaN,198 Old Hwy 10,...,ALABAMA,36782,Y,NaN,NaN,NaT,NaT,198 Old Highway 10,NaN,630973146.0
3,H-400-18215-374903,2018-10-01,H-2B,2018-09-04 09:00:07.433,Determination Issued - Certification,2018-12-01,2019-04-30,"Hamilton Harbor Marina, Inc.",Hamilton Harbor Yacht Club,7065 Hamilton Street,...,FLORIDA,34112,N,NaN,NaN,NaT,NaT,7065 Hamilton Street,NaN,593496970.0
4,H-400-18218-061822,2018-10-01,H-2B,2018-08-07 13:15:23.119,Determination Issued - Certification,2018-11-01,2019-04-01,OR Landscaping LLC,NaN,174 Spur Loop Road,...,WYOMING,83025,Y,NaN,NaN,NaT,NaT,6765 N. Crystal Springs Rd,NaN,453799159.0


First, let's only keep a subset of the relevant columns. Many of the columns are irrelevant or repetitious with regard to classification.

The columns that we're dropping (and reasons for dropping them) are explicitly stated in the analysis document.

In [39]:
RELEVANT_COLUMNS = [
    'CASE_STATUS', # Class label
    'AGENT_POC_EMP_REP_BY_AGENT',
    'SOC_CODE',
    'NAICS_CODE',
    'NBR_WORKERS_REQUESTED',
    'FULL_TIME_POSITION',
    'NATURE_OF_TEMPORARY_NEED',
    'BASIC_NUMBER_OF_HOURS',
    'BASIC_RATE_OF_PAY',
    'SUPERVISE_OTHER_EMP', # Needed temporarily to fill missing values in SUPERVISE_HOW_MANY with 0
    'SUPERVISE_HOW_MANY',
    'EDUCATION_LEVEL',
    'TRAINING_REQUIRED', # Needed temporarily to fill missing values in NUM_OF_MONTHS_TRAINING with 0
    'NUM_OF_MONTHS_TRAINING',
    'EMP_EXPERIENCE_REQD', # Needed temporarily to fill missing values in EMP_EXP_NUM_MONTHS with 0
    'EMP_EXP_NUM_MONTHS',
    'SWA_NAME',
    'AGENT_ATTORNEY_CITY', # Needed with WORKSITE_CITY for feature engineering CITY_MATCH
    'WORKSITE_CITY',
    'AGENT_ATTORNEY_STATE', # Needed with WORKSITE_STATE for feature engineering STATE_MATCH
    'WORKSITE_STATE',
    'HOURLY_WORK_SCHEDULE_AM', # Needed with HOURLY_WORK_SCHEDULE_PM for feature engineering DAYTIME_WORK and WORK_DAY_LENGTH 
    'HOURLY_WORK_SCHEDULE_PM',
    'OVERTIME_RATE_FROM', # Needed with OVERTIME_RATE_TO for feature engineering HAS_OVERTIME
    'OVERTIME_RATE_TO',
]

df = raw_data.loc[:, RELEVANT_COLUMNS]
print('Number of rows after data cleaning:', raw_data.shape[0])
print('Number of columns after data cleaning:', raw_data.shape[1])
df.head()

Number of rows after data cleaning: 8072
Number of columns after data cleaning: 62


,CASE_STATUS,AGENT_POC_EMP_REP_BY_AGENT,SOC_CODE,NAICS_CODE,NBR_WORKERS_REQUESTED,FULL_TIME_POSITION,NATURE_OF_TEMPORARY_NEED,BASIC_NUMBER_OF_HOURS,BASIC_RATE_OF_PAY,SUPERVISE_OTHER_EMP,...,EMP_EXP_NUM_MONTHS,SWA_NAME,AGENT_ATTORNEY_CITY,WORKSITE_CITY,AGENT_ATTORNEY_STATE,WORKSITE_STATE,HOURLY_WORK_SCHEDULE_AM,HOURLY_WORK_SCHEDULE_PM,OVERTIME_RATE_FROM,OVERTIME_RATE_TO
0,Determination Issued - Certification,Yes,37-3011,5617.0,5,Y,Peakload,40.0,13.68,No,...,NaN,NaN,BROOKLYN,SANDY,NEW YORK,UTAH,07:00:00,16:00:00,20.52,NaN
1,Determination Issued - Certification,Yes,45-4011,115310.0,49,Y,Seasonal,36.0,14.27,No,...,NaN,NaN,LAKE PARK,Hermitage,GEORGIA,ARKANSAS,06:00:00,13:00:00,21.41,22.50
2,Determination Issued - Partial Certification,Yes,45-4011,115310.0,130,Y,Seasonal,40.0,12.84,No,...,NaN,NaN,LAKE PARK,Sweet Water,GEORGIA,ALABAMA,08:00:00,17:00:00,19.26,25.76
3,Determination Issued - Certification,Yes,35-2014,713930.0,3,Y,Peakload,40.0,15.12,No,...,12.0,NaN,FRAMINGHAM,Naples,MASSACHUSETTS,FLORIDA,09:00:00,17:00:00,22.68,NaN
4,Determination Issued - Certification,Yes,37-3011,561730.0,10,Y,Seasonal,35.0,16.36,No,...,NaN,NaN,CHARLOTTESVILLE,Teton Village,VIRGINIA,WYOMING,05:00:00,13:00:00,24.54,NaN


Now, let's begin by performing some standardization on the class labels, indicated by the column `CASE_STATUS`. We will consider 'Determination Issued - Certification' as 'Certified' (numerical value of 1), and 'Determination Issued - Denied' and 'Determination Issued - Rejected' as 'Denied' (numerical value of 0), dropping all other records with differing (more ambiguous) class labels.

This makes the problem we're trying to solve a binary classification problem. Was the H2B application certified (accepted) or denied?

In [40]:
df.loc[df['CASE_STATUS'] == 'Determination Issued - Certification', 'CASE_STATUS'] = 1
df.loc[df['CASE_STATUS'] == 'Determination Issued - Denied', 'CASE_STATUS'] = 0
df.loc[df['CASE_STATUS'] == 'Determination Issued - Rejected', 'CASE_STATUS'] = 0
df = df[df['CASE_STATUS'].isin([1, 0])]

print("We retain {} rows after class label standardization.".format(df.shape[0]))
df['CASE_STATUS'].value_counts()

We retain 6743 rows after class label standardization.


1    5873
0     870
Name: CASE_STATUS, dtype: int64

The columns `SUPERVISE_HOW_MANY`, `NUM_OF_MONTHS_TRAINING`, and `EMP_EXP_NUM_MONTHS` contain missing values when the columns `SUPERVISE_OTHER_EMP`, `TRAINING_REQUIRED`, and `EMP_EXPERIENCE_REQD` have the value **No** respectively, so we first drop rows when there are missing values in `SUPERVISE_OTHER_EMP`, `TRAINING_REQUIRED`, and `EMP_EXPERIENCE_REQD`, and then we fill the values in `SUPERVISE_HOW_MANY`, `NUM_OF_MONTHS_TRAINING`, and `EMP_EXP_NUM_MONTHS` with **0** when there is a **No** in `SUPERVISE_OTHER_EMP`, `TRAINING_REQUIRED`, and `EMP_EXPERIENCE_REQD`.

In [41]:
df.dropna(subset=['SUPERVISE_OTHER_EMP', 'TRAINING_REQUIRED', 'EMP_EXPERIENCE_REQD'], inplace=True)
df.loc[df['SUPERVISE_OTHER_EMP'] == 'No', 'SUPERVISE_HOW_MANY'] = 0
df.loc[df['TRAINING_REQUIRED'] == 'No', 'NUM_OF_MONTHS_TRAINING'] = 0
df.loc[df['EMP_EXPERIENCE_REQD'] == 'No', 'EMP_EXP_NUM_MONTHS'] = 0

df.head()

,CASE_STATUS,AGENT_POC_EMP_REP_BY_AGENT,SOC_CODE,NAICS_CODE,NBR_WORKERS_REQUESTED,FULL_TIME_POSITION,NATURE_OF_TEMPORARY_NEED,BASIC_NUMBER_OF_HOURS,BASIC_RATE_OF_PAY,SUPERVISE_OTHER_EMP,...,EMP_EXP_NUM_MONTHS,SWA_NAME,AGENT_ATTORNEY_CITY,WORKSITE_CITY,AGENT_ATTORNEY_STATE,WORKSITE_STATE,HOURLY_WORK_SCHEDULE_AM,HOURLY_WORK_SCHEDULE_PM,OVERTIME_RATE_FROM,OVERTIME_RATE_TO
0,1,Yes,37-3011,5617.0,5,Y,Peakload,40.0,13.68,No,...,0.0,NaN,BROOKLYN,SANDY,NEW YORK,UTAH,07:00:00,16:00:00,20.52,NaN
1,1,Yes,45-4011,115310.0,49,Y,Seasonal,36.0,14.27,No,...,0.0,NaN,LAKE PARK,Hermitage,GEORGIA,ARKANSAS,06:00:00,13:00:00,21.41,22.5
3,1,Yes,35-2014,713930.0,3,Y,Peakload,40.0,15.12,No,...,12.0,NaN,FRAMINGHAM,Naples,MASSACHUSETTS,FLORIDA,09:00:00,17:00:00,22.68,NaN
4,1,Yes,37-3011,561730.0,10,Y,Seasonal,35.0,16.36,No,...,0.0,NaN,CHARLOTTESVILLE,Teton Village,VIRGINIA,WYOMING,05:00:00,13:00:00,24.54,NaN
5,1,Yes,37-2012,721110.0,12,Y,Peakload,35.0,12.66,No,...,6.0,NaN,FRAMINGHAM,West Dover,MASSACHUSETTS,VERMONT,08:00:00,15:00:00,18.99,NaN


Now that we've filled in missing data in `SUPERVISE_HOW_MANY`, `NUM_OF_MONTHS_TRAINING`, and `EMP_EXP_NUM_MONTHS`, we can drop the columns `SUPERVISE_OTHER_EMP`, `TRAINING_REQUIRED`, and `EMP_EXPERIENCE_REQD`.

In [42]:
df.drop(columns=['SUPERVISE_OTHER_EMP', 'TRAINING_REQUIRED', 'EMP_EXPERIENCE_REQD'], inplace=True)

print('Number of rows after dropping columns:', df.shape[0])
print('Number of columns after dropping columns:', df.shape[1])
df.head()

Number of rows after dropping columns: 6743
Number of columns after dropping columns: 22


,CASE_STATUS,AGENT_POC_EMP_REP_BY_AGENT,SOC_CODE,NAICS_CODE,NBR_WORKERS_REQUESTED,FULL_TIME_POSITION,NATURE_OF_TEMPORARY_NEED,BASIC_NUMBER_OF_HOURS,BASIC_RATE_OF_PAY,SUPERVISE_HOW_MANY,...,EMP_EXP_NUM_MONTHS,SWA_NAME,AGENT_ATTORNEY_CITY,WORKSITE_CITY,AGENT_ATTORNEY_STATE,WORKSITE_STATE,HOURLY_WORK_SCHEDULE_AM,HOURLY_WORK_SCHEDULE_PM,OVERTIME_RATE_FROM,OVERTIME_RATE_TO
0,1,Yes,37-3011,5617.0,5,Y,Peakload,40.0,13.68,0.0,...,0.0,NaN,BROOKLYN,SANDY,NEW YORK,UTAH,07:00:00,16:00:00,20.52,NaN
1,1,Yes,45-4011,115310.0,49,Y,Seasonal,36.0,14.27,0.0,...,0.0,NaN,LAKE PARK,Hermitage,GEORGIA,ARKANSAS,06:00:00,13:00:00,21.41,22.5
3,1,Yes,35-2014,713930.0,3,Y,Peakload,40.0,15.12,0.0,...,12.0,NaN,FRAMINGHAM,Naples,MASSACHUSETTS,FLORIDA,09:00:00,17:00:00,22.68,NaN
4,1,Yes,37-3011,561730.0,10,Y,Seasonal,35.0,16.36,0.0,...,0.0,NaN,CHARLOTTESVILLE,Teton Village,VIRGINIA,WYOMING,05:00:00,13:00:00,24.54,NaN
5,1,Yes,37-2012,721110.0,12,Y,Peakload,35.0,12.66,0.0,...,6.0,NaN,FRAMINGHAM,West Dover,MASSACHUSETTS,VERMONT,08:00:00,15:00:00,18.99,NaN


No rows were dropped after cleaning `SUPERVISE_HOW_MANY`, `NUM_OF_MONTHS_TRAINING`, and `EMP_EXP_NUM_MONTHS`.

Now, for the column `SWA_NAME`, we replace missing values with 0 and all other values with 1.
Although this is originally a categorical feature (containing the name of the State Workforce Agency supporting this job listing), we're performing a feat of feature engineering to convert this sparse categorical feature into a binary categorical feature.


This column will hold 0 if this job is not associated with a State Workforce Agency, and 1 if the job is associated with a State Workforce Agency.

In [43]:
print("Feature 'SWA_NAME' has {} possible values.".format(len(df['SWA_NAME'].value_counts())))

df.loc[df['SWA_NAME'].isna(), 'SWA_NAME'] = 0
df.loc[df['SWA_NAME'] != 0, 'SWA_NAME'] = 1
df['SWA_NAME'] = pd.to_numeric(df['SWA_NAME'])

print("Distribution of feature 'SWA_NAME' values after conversion into binary categorical feature: ")
print(df['SWA_NAME'].value_counts())
print('Number of rows remaining after conversion:', df.shape[0])

df.head()

Feature 'SWA_NAME' has 58 possible values.
Distribution of feature 'SWA_NAME' values after conversion into binary categorical feature: 
0    6676
1      67
Name: SWA_NAME, dtype: int64
Number of rows remaining after conversion: 6743


,CASE_STATUS,AGENT_POC_EMP_REP_BY_AGENT,SOC_CODE,NAICS_CODE,NBR_WORKERS_REQUESTED,FULL_TIME_POSITION,NATURE_OF_TEMPORARY_NEED,BASIC_NUMBER_OF_HOURS,BASIC_RATE_OF_PAY,SUPERVISE_HOW_MANY,...,EMP_EXP_NUM_MONTHS,SWA_NAME,AGENT_ATTORNEY_CITY,WORKSITE_CITY,AGENT_ATTORNEY_STATE,WORKSITE_STATE,HOURLY_WORK_SCHEDULE_AM,HOURLY_WORK_SCHEDULE_PM,OVERTIME_RATE_FROM,OVERTIME_RATE_TO
0,1,Yes,37-3011,5617.0,5,Y,Peakload,40.0,13.68,0.0,...,0.0,0,BROOKLYN,SANDY,NEW YORK,UTAH,07:00:00,16:00:00,20.52,NaN
1,1,Yes,45-4011,115310.0,49,Y,Seasonal,36.0,14.27,0.0,...,0.0,0,LAKE PARK,Hermitage,GEORGIA,ARKANSAS,06:00:00,13:00:00,21.41,22.5
3,1,Yes,35-2014,713930.0,3,Y,Peakload,40.0,15.12,0.0,...,12.0,0,FRAMINGHAM,Naples,MASSACHUSETTS,FLORIDA,09:00:00,17:00:00,22.68,NaN
4,1,Yes,37-3011,561730.0,10,Y,Seasonal,35.0,16.36,0.0,...,0.0,0,CHARLOTTESVILLE,Teton Village,VIRGINIA,WYOMING,05:00:00,13:00:00,24.54,NaN
5,1,Yes,37-2012,721110.0,12,Y,Peakload,35.0,12.66,0.0,...,6.0,0,FRAMINGHAM,West Dover,MASSACHUSETTS,VERMONT,08:00:00,15:00:00,18.99,NaN


Now, we want to clean OVERTIME_RATE_FROM and OVERTIME_RATE_TO.

First, for the column `OVERTIME_RATE_FROM`, we replace missing values with 0. 
Then for the column `OVERTIME_RATE_TO`, we replace missing values with the value in `OVERTIME_RATE_FROM`.

Eventually, when computing the binary categorical column HAS_OVERTIME in the feature engineering ipynb, this will allow us to more accurately ascertain if a job offers overtime pay, as we can simply check if either OVERTIME column contains a non-0 value / non-base-pay value.

In [44]:
df.loc[df['OVERTIME_RATE_FROM'].isna(), 'OVERTIME_RATE_FROM'] = 0
df.loc[df['OVERTIME_RATE_TO'].isna(), 'OVERTIME_RATE_TO'] = df['OVERTIME_RATE_FROM']

df.head()

,CASE_STATUS,AGENT_POC_EMP_REP_BY_AGENT,SOC_CODE,NAICS_CODE,NBR_WORKERS_REQUESTED,FULL_TIME_POSITION,NATURE_OF_TEMPORARY_NEED,BASIC_NUMBER_OF_HOURS,BASIC_RATE_OF_PAY,SUPERVISE_HOW_MANY,...,EMP_EXP_NUM_MONTHS,SWA_NAME,AGENT_ATTORNEY_CITY,WORKSITE_CITY,AGENT_ATTORNEY_STATE,WORKSITE_STATE,HOURLY_WORK_SCHEDULE_AM,HOURLY_WORK_SCHEDULE_PM,OVERTIME_RATE_FROM,OVERTIME_RATE_TO
0,1,Yes,37-3011,5617.0,5,Y,Peakload,40.0,13.68,0.0,...,0.0,0,BROOKLYN,SANDY,NEW YORK,UTAH,07:00:00,16:00:00,20.52,20.52
1,1,Yes,45-4011,115310.0,49,Y,Seasonal,36.0,14.27,0.0,...,0.0,0,LAKE PARK,Hermitage,GEORGIA,ARKANSAS,06:00:00,13:00:00,21.41,22.50
3,1,Yes,35-2014,713930.0,3,Y,Peakload,40.0,15.12,0.0,...,12.0,0,FRAMINGHAM,Naples,MASSACHUSETTS,FLORIDA,09:00:00,17:00:00,22.68,22.68
4,1,Yes,37-3011,561730.0,10,Y,Seasonal,35.0,16.36,0.0,...,0.0,0,CHARLOTTESVILLE,Teton Village,VIRGINIA,WYOMING,05:00:00,13:00:00,24.54,24.54
5,1,Yes,37-2012,721110.0,12,Y,Peakload,35.0,12.66,0.0,...,6.0,0,FRAMINGHAM,West Dover,MASSACHUSETTS,VERMONT,08:00:00,15:00:00,18.99,18.99


Finally, we want to clean the values associated with the AGENT_ATTORNEY_CITY and AGENT_ATTORNEY_STATE columns.
The problem is that these columns have many NaN values associated with rows that have "no" for [AGENT_POC_EMP_REP_BY_AGENT], representing no attorney representation.

To resolve this problem, we'll define a custom value "no attorney representation" to fill for these column values associated with no attorney representation.

In [45]:
# Let's define a custom value to use here to avoid the problem of NaN.
NO_ATTORNEY_REPRESENTATION = "no attorney representation"

# Just to check that it's not currently in use for these column values:
assert NO_ATTORNEY_REPRESENTATION not in df['AGENT_ATTORNEY_CITY'].values
assert NO_ATTORNEY_REPRESENTATION not in df['AGENT_ATTORNEY_STATE'].values

df.loc[df['AGENT_POC_EMP_REP_BY_AGENT'] == "No", 'AGENT_ATTORNEY_CITY'] = NO_ATTORNEY_REPRESENTATION
df.loc[df['AGENT_POC_EMP_REP_BY_AGENT'] == "No", 'AGENT_ATTORNEY_STATE'] = NO_ATTORNEY_REPRESENTATION

Finally, we want to deal with a few small NaN edge cases that can be easily dealt with.

We can also check how many rows that we'll end up eliminating that have missing values.

In [46]:
df.loc[df['EMP_EXP_NUM_MONTHS'].isna(), 'EMP_EXP_NUM_MONTHS'] = 0
df.loc[df['SUPERVISE_HOW_MANY'].isna(), 'SUPERVISE_HOW_MANY'] = 0
df_with_nan = df[df.isna().any(axis=1)]
print("Remaining rows with NaN values: " + str(len(df_with_nan)))

Remaining rows with NaN values: 37


Now, we perform listwise deletion on all rows that still contain missing values, and rows that are duplicates. We drop duplicate rows since these most likely represent multiple H2B visa requests by the same entity for the same job.

In [47]:
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)
print('Number of rows after dropping rows with missing values / duplicates:', df.shape[0])

Number of rows after dropping rows with missing values / duplicates: 6540


In (categorical) columns that have Yes / No values, we replace the values with 1 / 0.

In [48]:
df['AGENT_POC_EMP_REP_BY_AGENT'] = df['AGENT_POC_EMP_REP_BY_AGENT'].map({'Yes': 1, 'No': 0})
df['FULL_TIME_POSITION'] = df['FULL_TIME_POSITION'].map({'Y': 1, 'N': 0})
df.head()

,CASE_STATUS,AGENT_POC_EMP_REP_BY_AGENT,SOC_CODE,NAICS_CODE,NBR_WORKERS_REQUESTED,FULL_TIME_POSITION,NATURE_OF_TEMPORARY_NEED,BASIC_NUMBER_OF_HOURS,BASIC_RATE_OF_PAY,SUPERVISE_HOW_MANY,...,EMP_EXP_NUM_MONTHS,SWA_NAME,AGENT_ATTORNEY_CITY,WORKSITE_CITY,AGENT_ATTORNEY_STATE,WORKSITE_STATE,HOURLY_WORK_SCHEDULE_AM,HOURLY_WORK_SCHEDULE_PM,OVERTIME_RATE_FROM,OVERTIME_RATE_TO
0,1,1,37-3011,5617.0,5,1,Peakload,40.0,13.68,0.0,...,0.0,0,BROOKLYN,SANDY,NEW YORK,UTAH,07:00:00,16:00:00,20.52,20.52
1,1,1,45-4011,115310.0,49,1,Seasonal,36.0,14.27,0.0,...,0.0,0,LAKE PARK,Hermitage,GEORGIA,ARKANSAS,06:00:00,13:00:00,21.41,22.50
3,1,1,35-2014,713930.0,3,1,Peakload,40.0,15.12,0.0,...,12.0,0,FRAMINGHAM,Naples,MASSACHUSETTS,FLORIDA,09:00:00,17:00:00,22.68,22.68
4,1,1,37-3011,561730.0,10,1,Seasonal,35.0,16.36,0.0,...,0.0,0,CHARLOTTESVILLE,Teton Village,VIRGINIA,WYOMING,05:00:00,13:00:00,24.54,24.54
5,1,1,37-2012,721110.0,12,1,Peakload,35.0,12.66,0.0,...,6.0,0,FRAMINGHAM,West Dover,MASSACHUSETTS,VERMONT,08:00:00,15:00:00,18.99,18.99


Then, we standardize all of the string type categorical value columns to be lowercase.

In [49]:
df['NATURE_OF_TEMPORARY_NEED'] = df['NATURE_OF_TEMPORARY_NEED'].str.lower()
df['EDUCATION_LEVEL'] = df['EDUCATION_LEVEL'].str.lower()
df['WORKSITE_STATE'] = df['WORKSITE_STATE'].str.lower()
df['AGENT_ATTORNEY_CITY'] = df['AGENT_ATTORNEY_CITY'].str.lower()
df['WORKSITE_CITY'] = df['WORKSITE_CITY'].str.lower()
df['AGENT_ATTORNEY_STATE'] = df['AGENT_ATTORNEY_STATE'].str.lower()
df.head()

,CASE_STATUS,AGENT_POC_EMP_REP_BY_AGENT,SOC_CODE,NAICS_CODE,NBR_WORKERS_REQUESTED,FULL_TIME_POSITION,NATURE_OF_TEMPORARY_NEED,BASIC_NUMBER_OF_HOURS,BASIC_RATE_OF_PAY,SUPERVISE_HOW_MANY,...,EMP_EXP_NUM_MONTHS,SWA_NAME,AGENT_ATTORNEY_CITY,WORKSITE_CITY,AGENT_ATTORNEY_STATE,WORKSITE_STATE,HOURLY_WORK_SCHEDULE_AM,HOURLY_WORK_SCHEDULE_PM,OVERTIME_RATE_FROM,OVERTIME_RATE_TO
0,1,1,37-3011,5617.0,5,1,peakload,40.0,13.68,0.0,...,0.0,0,brooklyn,sandy,new york,utah,07:00:00,16:00:00,20.52,20.52
1,1,1,45-4011,115310.0,49,1,seasonal,36.0,14.27,0.0,...,0.0,0,lake park,hermitage,georgia,arkansas,06:00:00,13:00:00,21.41,22.50
3,1,1,35-2014,713930.0,3,1,peakload,40.0,15.12,0.0,...,12.0,0,framingham,naples,massachusetts,florida,09:00:00,17:00:00,22.68,22.68
4,1,1,37-3011,561730.0,10,1,seasonal,35.0,16.36,0.0,...,0.0,0,charlottesville,teton village,virginia,wyoming,05:00:00,13:00:00,24.54,24.54
5,1,1,37-2012,721110.0,12,1,peakload,35.0,12.66,0.0,...,6.0,0,framingham,west dover,massachusetts,vermont,08:00:00,15:00:00,18.99,18.99


Now, we perform standardization on the columns 'SOC_CODE' and 'NAICS_CODE'.
First, we remove dashes from SOC_CODE.

Then, since both `SOC_CODE` and `NAICS_CODE` have a large number of unique values, and this could result in a sparse dataset when performing one-hot encoding on these categorical values, we trim both `SOC_CODE` and `NAICS_CODE` to their first 2 digits. Keeping the first 2 digits preserves broad topic information about certain industries and occupations, while also reducing the diversity of values in those columns. More information can be found in our report.

As such, we standardize the columns `SOC_CODE` and `NAICS_CODE` to store integers that are 2 digits long.

In [50]:
df['SOC_CODE'] = df['SOC_CODE'].str.replace('-', '', regex=False)
df['NAICS_CODE'] = df['NAICS_CODE'].astype('str')

print("Before cleaning, SOC_CODE has {} possible values.".format(len(df["SOC_CODE"].value_counts())))
print("Before cleaning, NAICS_CODE has {} possible values.".format(len(df["NAICS_CODE"].value_counts())))

def trim_to_2_digits(row):
    return (row["NAICS_CODE"][0:2], row["SOC_CODE"][0:2])

df[["NAICS_CODE", "SOC_CODE"]] = df.apply(trim_to_2_digits, axis=1, result_type='expand')

df['SOC_CODE'] = pd.to_numeric(df['SOC_CODE'])
df['SOC_CODE'] = df['SOC_CODE'].astype('int64')
df['NAICS_CODE'] = df['NAICS_CODE'].astype('int64')

print("After cleaning, SOC_CODE has {} possible values.".format(len(df["SOC_CODE"].value_counts())))
print("After cleaning, NAICS_CODE has {} possible values.".format(len(df["NAICS_CODE"].value_counts())))

df.head()

Before cleaning, SOC_CODE has 185 possible values.
Before cleaning, NAICS_CODE has 343 possible values.
After cleaning, SOC_CODE has 20 possible values.
After cleaning, NAICS_CODE has 24 possible values.


,CASE_STATUS,AGENT_POC_EMP_REP_BY_AGENT,SOC_CODE,NAICS_CODE,NBR_WORKERS_REQUESTED,FULL_TIME_POSITION,NATURE_OF_TEMPORARY_NEED,BASIC_NUMBER_OF_HOURS,BASIC_RATE_OF_PAY,SUPERVISE_HOW_MANY,...,EMP_EXP_NUM_MONTHS,SWA_NAME,AGENT_ATTORNEY_CITY,WORKSITE_CITY,AGENT_ATTORNEY_STATE,WORKSITE_STATE,HOURLY_WORK_SCHEDULE_AM,HOURLY_WORK_SCHEDULE_PM,OVERTIME_RATE_FROM,OVERTIME_RATE_TO
0,1,1,37,56,5,1,peakload,40.0,13.68,0.0,...,0.0,0,brooklyn,sandy,new york,utah,07:00:00,16:00:00,20.52,20.52
1,1,1,45,11,49,1,seasonal,36.0,14.27,0.0,...,0.0,0,lake park,hermitage,georgia,arkansas,06:00:00,13:00:00,21.41,22.50
3,1,1,35,71,3,1,peakload,40.0,15.12,0.0,...,12.0,0,framingham,naples,massachusetts,florida,09:00:00,17:00:00,22.68,22.68
4,1,1,37,56,10,1,seasonal,35.0,16.36,0.0,...,0.0,0,charlottesville,teton village,virginia,wyoming,05:00:00,13:00:00,24.54,24.54
5,1,1,37,72,12,1,peakload,35.0,12.66,0.0,...,6.0,0,framingham,west dover,massachusetts,vermont,08:00:00,15:00:00,18.99,18.99


Now that data cleaning is completed, we output the dataframe to a new CSV.

In [51]:
print('Number of rows:', df.shape[0])
print('Number of columns:', df.shape[1])
df.to_csv('H-2B_Cleaned_Data.csv', index=False)

Number of rows: 6540
Number of columns: 22
